## Create Flood Inundation Map based on the HAND Method

**Authors**: Anthony Castronova (acastronova@cuahsi.org), Irene Garousi-Nejad (igarousi@cuahsi.org)  <br>
**Last updated**: Mar 26, 2024

This notebook demonstrates how to generate flood inundation maps using the methodology and datasets defined by https://github.com/noAA-OWP/inundation-mapping. All data used in this notebook are publicly acessible via Amazon AWS. 

This work was funded by:

 <img src="https://www.hydroshare.org/resource/dc269e23ff494a06b7372bc6034a5de2/data/contents/v3-SNOW/logo-img/CUAHSI-4-color-logo_with_URL.png" width="300" height="300" style="padding-right:50px"> <img src="https://www.hydroshare.org/resource/dc269e23ff494a06b7372bc6034a5de2/data/contents/v3-SNOW/logo-img/CIROHLogo_200x200.png" width="100" height="20">    

In [ ]:
import numpy
import xarray
import rioxarray
import geopandas
import numpy as np
import pandas as pd
from typing import Dict
from pathlib import Path
from scipy import interpolate
import matplotlib.pyplot as plt
from geocube.api.core import make_geocube

## Collect Data

Collect the required input datasets for the FIM computation, we've staged some for public access in the CUAHSI cloud. For access to all available FIM input datasets, please refer to the documentation at https://github.com/noAA-OWP/inundation-mapping. 

In [ ]:
# hydroTable_0.csv
!aws --no-sign-request --endpoint-url https://api.minio.cuahsi.io \
 s3 cp s3://scratch/hand_fim/07010108/branches/0/hydroTable_0.csv .

# rem_zeroed_masked_0.tif
!aws --no-sign-request --endpoint-url https://api.minio.cuahsi.io \
 s3 cp s3://scratch/hand_fim/07010108/branches/0/rem_zeroed_masked_0.tif .

# gw_catchments_reaches_filtered_addedAttributes_crosswalked_0.gpkg
!aws --no-sign-request --endpoint-url https://api.minio.cuahsi.io \
s3 cp s3://scratch/hand_fim/07010108/branches/0/gw_catchments_reaches_filtered_addedAttributes_crosswalked_0.gpkg .

Define a feature of interest and a hypothetical streamflow

In [ ]:
# feature id of interest
nhd_feature_id=4966269

# hypothetical streamflow (cms).
# This can be replaced with a flow value gathered by a source of your choice
cms = 85

Use the streamflow defined above to interpolate river stage from a rating curve for all hydroids that exist within this NHD+ reach. 

In [ ]:
def interpolate_y(df: pd.DataFrame,
                  x_column: str,
                  y_column: str,
                  x_value: float) -> float:
    """
    Performs 1D interpolation on two columns of a Dataframe.

    Parameters
    ==========
    df: pandas.DataFrame
        DataFrame containing data that will be used in the interpolation.
    x_column: str
        Name of the column that represents the X-axis data.
    y_column: str
        Name of the column that represents the Y-axis data.
    x_value: float
        Numeric X-axis value for which to interpolate Y-axis data. Returns 
        -9999 if the interpolation fails to resolve.

    Returns
    =======
    y_value: float
        Numeric Y-axis value corresponding to the input X-axis value.

    """
    # Sort the DataFrame by the 'x' column to ensure interpolation works correctly
    df_sorted = df.sort_values(by=x_column)
    
    # Check if the x_value is within the range of the DataFrame
    if x_value < df_sorted[x_column].min() or x_value > df_sorted[x_column].max():
        return -9999  # x_value is out of range, cannot interpolate
    
    # Perform linear interpolation
    f = interpolate.interp1d(df_sorted[x_column], df_sorted[y_column], kind='linear')
    
    # Return the interpolated y value for the given x value
    return f(x_value)

def compute_stage(df: pd.DataFrame,
                  hydro_id: int,
                  flow_cms: float) -> Dict[int, float]:
    """
    Computes river stage from a rating curve given streamflow in cfs.

    Parameters
    ==========
    df: pandas.DataFrame
        DataFrame containing the stage and discharge values of the rating curve.
        This must contain the following columns: HydroID, stage, discharge_cms.
    hydro_id: int
        Identifier for the reach for which to compute stage.
    flow_cms: float
        Streamflow to convert into river stage.

    Returns
    =======
    Dict [int, float]
        A dictionary containing computed stage and its associated hydroid

    """

    
    # look up rating curve for this hydroid
    rating_curve = df.loc[df.HydroID == hydro_id, ['stage', 'discharge_cms']]

    # interpolate using the provided flow rate
    interpolated_stage = interpolate_y(rating_curve, 'discharge_cms', 'stage', flow_cms) 

    return {hydro_id: float(interpolated_stage)}

def get_stage_for_all_hydroids_in_reach(nhd_feature_id: int,
                                        flow_cms: float,
                                        hydrotable: Path = Path('./hydroTable_0.csv')) -> Dict[int, float]:
    """
    Retrieves stage for all NWM HydroIDs given an NHD reach and input streamflow.
    The stage is computed using FIM rating curves.

    Parameters
    ==========
    nhd_feature_id: int
        NHD feature identifier.
    flow_cms: float
        Streamflow for the reach in cubic meters per second
    hydrotable: pathlib.Path
        Path to the FIM hydrotable.csv file containing rating curve data.

    Returns
    =======
    Dict [int, float]
        Dictionary containing one or more NWM hydro identifiers and 
        their corresponding stages.

    """
    
    # load hydrotable_0
    # we don't need all of the columns in this csv
    hydro_df = pd.read_csv(hydrotable,
                           usecols=['HydroID', 'NextDownID', 'feature_id',
                                    'stage', 'discharge_cms'])
    
    # select features that match nhd_feature_id
    d = hydro_df.loc[hydro_df.feature_id==nhd_feature_id]
    
    # get unique combos of HydroID and NextDownID 
    hydro_ids = np.unique(d.HydroID)
    
    interpolated_stages = {}
    for hydro_id in hydro_ids:  
        interpolated_stages.update(compute_stage(d, hydro_id, flow_cms))
        
    # return interpolated stage
    return interpolated_stages


In [ ]:
stage_dict = get_stage_for_all_hydroids_in_reach(nhd_feature_id, cms)

In [ ]:
stage_dict

Load the precomputed HAND raster that was obtained from AWS.

In [ ]:
xds = rioxarray.open_rasterio(Path('./rem_zeroed_masked_0.tif'),
                              masked=True).squeeze().drop_vars('band').to_dataset(name='hand')
xds

Make a copy of the 'hand' variable to the 'stage' variable. This will be used to compute the flood inundation map later on. The dataset will now have both `hand` and `stage` variables for all (x,y) locations.

In [ ]:
xds['stage'] = xds.hand.copy(deep=True)
xds

Read watershed geometries and set stage values from the `stage_dict` defined above, and remove all other geometries. The end product will be a geodataframe containing the geometries for each `hydroid` in our area of interest and their stage values.

In [ ]:
geodf = geopandas.read_file(Path('./gw_catchments_reaches_filtered_addedAttributes_crosswalked_0.gpkg'))

# loop over each reach and set the corresponding stage
# in the geopandas object
for hydroid, stage in stage_dict.items():
    geodf.loc[geodf.HydroID==hydroid, 'stage'] = stage

# remove all nan values to make our dataset smaller
geodf_filtered = geodf[geodf.stage.notnull()]

geodf_filtered

Visualize the area that we're working with.

In [ ]:
figure, ax = plt.subplots(1, figsize=(10, 10))

geodf.plot(facecolor='None', edgecolor='lightgrey', ax=ax);
geodf_filtered.plot(facecolor='green', ax=ax);

Add all geometries to the Dataset containing `hand` and `stage` variables using a GeoCube.

In [ ]:
# create a grid for the geocube
out_grid = make_geocube(
    vector_data=geodf,
    measurements=['HydroID'],
    like=xds # ensure the data are on the same grid
)

# add stage and hydroID to the HAND raster
#xds = xds.assign_coords( hydroid = (['y', 'x'], out_grid.HydroID.data) )
ds = xds.assign_coords( hydroid = (['y', 'x'], out_grid.HydroID.data) )

# drop everything except the HydroIDs that we're interested in
ds = ds.where(ds.hydroid.isin(geodf_filtered.HydroID), drop=True)
ds

Update the stage values in the DataSet where specific hydroid's exist.

In [ ]:
for idx, row in geodf_filtered.iterrows():
    print(f'{row.HydroID} -> {row.stage}' )
    ds['stage'] = xarray.where(ds.hydroid == row.HydroID, row.stage, ds.stage)

Preview the `stage` data we set as well as the original `hand` data. 

In [ ]:
figure, axes = plt.subplots(1, 2, figsize=(10,5))

# plot stage
ds.stage.plot(ax=axes[0])
axes[0].set_title('Stage');
axes[0].tick_params(axis='x', labelrotation=45)

# plot hand
ds.hand.plot(ax=axes[1])
axes[1].set_title('HAND');
axes[1].tick_params(axis='x', labelrotation=45)

figure.tight_layout()


Compute FIM by subtracting `hand` from `stage`. Everything that is negative should be set to zero. Cells that have a value greater than zero indicate areas in which flooding occurs. Create a new variable in our dataset (called `fim`) to store this result.

In [ ]:
ds['fim'] = ds.stage - ds.hand
ds['fim'] = xarray.where(ds.fim >= 0.00001, ds.fim, numpy.nan)

In [ ]:
figure, axes = plt.subplots(1, 2, figsize=(12,5))

geodf.loc[geodf.feature_id==nhd_feature_id].plot(ax=axes[0], edgecolor='grey', facecolor='None');
ds.fim.plot(cbar_kwargs={'label':'depth [meters]'}, ax=axes[0]);
axes[0].set_title(f'FIM for NHD {nhd_feature_id} at {cms} cms');


xarray.plot.contourf(ds.fim, levels=4, cmap='Blues', cbar_kwargs={'label':'Flood Risk (light = low risk)'}, ax=axes[1]);
geodf.loc[geodf.feature_id==nhd_feature_id].plot(ax=axes[1], edgecolor='grey', facecolor='None');
axes[1].set_title(f'FIM Contours for NHD {nhd_feature_id} at {cms} cms');
